In [24]:
COS = "Courses-of-Study_2020-2021.pdf"
COURSE_DESCRIPTION_PAGE_FIRST = 161
COURSE_DESCRIPTION_PAGE_LAST = 370

In [13]:
import fitz  # PyMuPDF
import re

In [14]:
COURSE_PATTERN = r"^[A-Z]{3}\d{3}"
CREDIT_PATTERN = r"^\d"
PREREQ_PATTERN = r"^Pre-requi"
OVERLAP_PATTERN = r"^Overlaps"

In [25]:
def extract_text_by_coordinates(pdf_document, page_number, x1, y1, x2, y2):
    page = pdf_document[page_number - 1]
    rect = fitz.Rect(x1, y1, x2, y2)
    
    selected_text = page.get_text("text", clip=rect)
    return selected_text

def standard_cos_page_scraper(pdf_doc, page):
    x1, y1 = 49, 84  # Starting coordinates of the rectangle
    x2, y2 = 300, 790  # Ending coordinates of the rectangle
    x3, y3 = 300, 84
    x4, y4 = 596, 790
    left_side = extract_text_by_coordinates(pdf_doc, page, x1, y1, x2, y2)
    right_side = extract_text_by_coordinates(pdf_doc, page, x3, y3, x4, y4)
    return [left_side, right_side]


def scrape_courses_description():
    scraped_dataset = []
    
    first_page_i = COURSE_DESCRIPTION_PAGE_FIRST
    last_page_i = COURSE_DESCRIPTION_PAGE_LAST


    pdf_doc = fitz.open(COS)
    for page in range(first_page_i, last_page_i + 1):
        scraped_dataset.extend(standard_cos_page_scraper(pdf_doc, page))

    pdf_doc.close()
    
    return scraped_dataset

def attach_courses_description(scraped_dataset):
    for i in range(len(scraped_dataset)):
        data = scraped_dataset[i]
        if re.match(COURSE_PATTERN, data):
            scraped_dataset[i] = "\n" + data
        else :
            scraped_dataset[i] = " " + data
    return "".join(scraped_dataset)

def string_course_dataset_to_json(single_string_dataset):
    all_lines = single_string_dataset.split("\n")[1:]
    all_courses = []
    course = {}
    is_first_course = True
    for line in all_lines:
        if re.match(COURSE_PATTERN, line):
            if is_first_course: is_first_course = False
            else: all_courses.append(course)
            course = {
                "code" : line[:6],
                "name" : line[7:],
                "description" : "",
                "preReq" : "",
                "overLap" : "",
                "credits" : ""
            }
        elif re.match(CREDIT_PATTERN, line): course["credits"] = line
        elif re.match(PREREQ_PATTERN, line): course["preReq"] = line
        elif re.match(OVERLAP_PATTERN, line): course["overLap"] = line
        else: course["description"]  += line
    all_courses.append(course)

    return all_courses



def get_all_courses_description():
    scraped_dataset : list[str] = scrape_courses_description()
    single_string_dataset : str = attach_courses_description(scraped_dataset)
    courses_json_list : list[object] = string_course_dataset_to_json(single_string_dataset)
    return courses_json_list

print(get_all_courses_description())
    

[{'code': 'APL100', 'name': 'Engineering Mechanics', 'description': 'Kinematics, Statics, Equations of Motion, Rigid body dynamics, Introduction to variational mechanics.', 'preReq': '', 'overLap': '', 'credits': '4 Credits (3-1-0)'}, {'code': 'APL101', 'name': 'Applied Mathematics in Engineering ', 'description': 'Applications ordinary Differential Equation: Second order oDEs, Method of Undetermined Coefficients, Variation of Parameters, Strum-Liouville eigenvalue problem, Difference equation. Partial Differential Equation: Classification of PDEs, Heat, Wave and Laplace Equations, Separation of variables to solve PDEs. Fourier Transform: Fourier sine transform, Fourier cosine Transform, Technique for solving oDEs and PDEs. Probability Theory: Axioms of probability, Conditional probability, Random variable, uncertainty in engineering system, Discrete and Continuous distributions, Distribution function, Joint probability distribution, Moments,  Covariance, Correlation coefficient. Stoch